In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
# Wiki URL
en_url = "https://en.wikipedia.org/wiki/List_of_largest_banks_in_the_United_States"

In [3]:
# Openning connection and request page
uClient = uReq(en_url)
page_html = uClient.read()
uClient.close()

In [4]:
# Parse HTML
page_soup = soup(page_html, "html.parser")

In [5]:
# Grab each row
containers = page_soup.findAll("tr")

In [6]:
len(containers)

108

In [7]:
# Now we check what are in containers
containers[0]

<tr>
<th data-sort-type="number">Rank
</th>
<th>Bank name
</th>
<th>Headquarters location
</th>
<th>Total assets (billions of US$)<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup>
</th>
<th>Market capitalization (billions of US$)<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup>
</th></tr>

In [8]:
containers[1]

<tr>
<td>1
</td>
<td><a href="/wiki/JPMorgan_Chase" title="JPMorgan Chase">JPMorgan Chase</a>
</td>
<td><a href="/wiki/New_York_City" title="New York City">New York City</a>
</td>
<td>$2,590
</td>
<td>$384
</td></tr>

In [9]:
# We only want the banks, so we need some list slicing
banks = containers[1:101]

In [10]:
# Create a file to hold information
df = pd.DataFrame(columns=["Rank", "Bank Name", "Headquarter Location", 
                           "Total Assets (Billions of USD)", 
                           "Market Capitalization (Billions of USD)"])
df

,Rank,Bank Name,Headquarter Location,Total Assets (Billions of USD),Market Capitalization (Billions of USD)


In [11]:
f = open("commands.txt", "w")

In [12]:
for i in range(len(banks)):
    bank = banks[i]
    rank = int(bank.td.text)
    
    name = bank.findAll('td')[1].a["title"]
    name = list(name)
    if '\'' in name:
        name.remove('\'')
    name = ''.join(name[:39])
    
    headquarter = bank.findAll('td')[2].a["title"]
    
    asset = list(bank.findAll('td')[3].text[1:-1])
    if "," in asset:
        asset.remove(",")
    asset = int(''.join(asset))
    asset
    
    mc = list(bank.findAll('td')[4].text[1:-1])
    if "," in mc:
        mc.remove(",")
    if "A" in mc or len(mc) == 0:
        mc = -1
    else:
        mc = int(''.join(mc))
        
    f.write("INSERT INTO banks VALUE (%s, '%s', '%s', %s, %s);\n" %(rank, name, headquarter, asset, mc))
    df.loc[i] = [rank, name, headquarter, asset, mc]

f.close()

In [13]:
df.head()

,Rank,Bank Name,Headquarter Location,Total Assets (Billions of USD),Market Capitalization (Billions of USD)
0,1,JPMorgan Chase,New York City,2590,384
1,2,Bank of America,"Charlotte, North Carolina",2291,302
2,3,Citigroup,New York City,1912,174
3,4,Wells Fargo,San Francisco,1879,273
4,5,Goldman Sachs,New York City,968,87


In [14]:
df.to_csv("banks.csv", index = False)

In [15]:
pd.read_csv("banks.csv").head()

,Rank,Bank Name,Headquarter Location,Total Assets (Billions of USD),Market Capitalization (Billions of USD)
0,1,JPMorgan Chase,New York City,2590,384
1,2,Bank of America,"Charlotte, North Carolina",2291,302
2,3,Citigroup,New York City,1912,174
3,4,Wells Fargo,San Francisco,1879,273
4,5,Goldman Sachs,New York City,968,87
